In [3]:
import numpy as np
import os
from tensorflow.keras.applications import InceptionV3
from tensorflow.keras.preprocessing import image
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.optimizers import Adam
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_curve, roc_auc_score
from skimage.transform import resize
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder
import numpy as np


In [4]:
# Function to load images and labels from folder
def load_images_and_labels(folder, target_size=(100, 100)):
    images = []
    labels = []
    for subfolder in os.listdir(folder):
        subfolder_path = os.path.join(folder, subfolder)
        label = subfolder
        for filename in os.listdir(subfolder_path):
            image_path = os.path.join(subfolder_path, filename)
            try:
                img = image.load_img(image_path, target_size=target_size)
                img = image.img_to_array(img)
                img = img / 255.0  # Normalize pixel values to [0, 1]
                images.append(img)
                labels.append(label)
            except Exception as e:
                print(f"Error loading image: {image_path} - {str(e)}")
    return np.array(images), np.array(labels)

In [5]:
# Folder paths
train_folder = 'C:/Users/KIIT/Downloads/Cashew/Cashew/train'
test_folder ='C:/Users/KIIT/Downloads/Cashew/Cashew/test'
val_folder ='C:/Users/KIIT/Downloads/Cashew/Cashew/val'

In [6]:
# Load images and labels for training, testing, and validation
X_train, y_train = load_images_and_labels(train_folder)
X_test, y_test = load_images_and_labels(test_folder)
X_val, y_val = load_images_and_labels(val_folder)


In [7]:
# Convert string labels to integer labels
label_encoder = LabelEncoder()
y_train = label_encoder.fit_transform(y_train)
y_test = label_encoder.transform(y_test)
y_val = label_encoder.transform(y_val)

In [8]:
# Load InceptionV3 model pre-trained on ImageNet without the top classification layer
base_model = InceptionV3(weights='imagenet', include_top=False)

In [9]:
# Add a global spatial average pooling layer
x = base_model.output
x = GlobalAveragePooling2D()(x)

In [10]:
# Add a fully-connected layer
x = Dense(1024, activation='relu')(x)


In [11]:
# Add a logistic layer for the number of classes
predictions = Dense(len(set(y_train)), activation='softmax')(x)

In [12]:
# Model
model = Model(inputs=base_model.input, outputs=predictions)


In [13]:
# Compile the model
model.compile(optimizer=Adam(learning_rate=0.0001), loss='sparse_categorical_crossentropy', metrics=['accuracy'])


In [14]:
# Train the model
model.fit(X_train, y_train, epochs=10, batch_size=32, validation_data=(X_val, y_val))


Epoch 1/10
201/201 ━━━━━━━━━━━━━━━━━━━━ 242s 1s/step - accuracy: 0.5847 - loss: 0.9868 - val_accuracy: 0.8842 - val_loss: 0.3288
Epoch 2/10
201/201 ━━━━━━━━━━━━━━━━━━━━ 196s 976ms/step - accuracy: 0.8890 - loss: 0.3305 - val_accuracy: 0.9066 - val_loss: 0.2689
Epoch 3/10
201/201 ━━━━━━━━━━━━━━━━━━━━ 190s 947ms/step - accuracy: 0.9308 - loss: 0.1936 - val_accuracy: 0.9128 - val_loss: 0.2694
Epoch 4/10
201/201 ━━━━━━━━━━━━━━━━━━━━ 207s 1s/step - accuracy: 0.9513 - loss: 0.1617 - val_accuracy: 0.9290 - val_loss: 0.2781
Epoch 5/10
201/201 ━━━━━━━━━━━━━━━━━━━━ 191s 950ms/step - accuracy: 0.9602 - loss: 0.1241 - val_accuracy: 0.9340 - val_loss: 0.2600
Epoch 6/10
201/201 ━━━━━━━━━━━━━━━━━━━━ 205s 1s/step - accuracy: 0.9788 - loss: 0.0772 - val_accuracy: 0.9240 - val_loss: 0.2993
Epoch 7/10
201/201 ━━━━━━━━━━━━━━━━━━━━ 202s 1s/step - accuracy: 0.9791 - loss: 0.0770 - val_accuracy: 0.9315 - val_loss: 0.3396
Epoch 8/10
201/201 ━━━━━━━━━━━━━━━━━━━━ 188s 936ms/step - accuracy: 0.9887 - loss: 0.040

In [15]:
# Predictions
train_pred = model.predict(X_train)
val_pred = model.predict(X_val)
test_pred = model.predict(X_test)

201/201 ━━━━━━━━━━━━━━━━━━━━ 37s 170ms/step
26/26 ━━━━━━━━━━━━━━━━━━━━ 5s 175ms/step
25/25 ━━━━━━━━━━━━━━━━━━━━ 4s 166ms/step


In [16]:
# Predict probability estimates
train_pred_proba = model.predict(X_train)
val_pred_proba = model.predict(X_val)
test_pred_proba = model.predict(X_test)

# Get probability estimates for the positive class
train_pos_proba = train_pred_proba[:, 1]
val_pos_proba = val_pred_proba[:, 1]
test_pos_proba = test_pred_proba[:, 1]

201/201 ━━━━━━━━━━━━━━━━━━━━ 29s 143ms/step
26/26 ━━━━━━━━━━━━━━━━━━━━ 5s 181ms/step
25/25 ━━━━━━━━━━━━━━━━━━━━ 4s 153ms/step


In [17]:

# Evaluation Metrics 
train_accuracy = accuracy_score(y_train, np.argmax(train_pred, axis=1))
val_accuracy = accuracy_score(y_val, np.argmax(val_pred, axis=1))
test_accuracy = accuracy_score(y_test, np.argmax(test_pred, axis=1))

train_precision = precision_score(y_train, np.argmax(train_pred, axis=1), average='weighted')
val_precision = precision_score(y_val, np.argmax(val_pred, axis=1), average='weighted')
test_precision = precision_score(y_test, np.argmax(test_pred, axis=1), average='weighted')

train_recall = recall_score(y_train, np.argmax(train_pred, axis=1), average='weighted')
val_recall = recall_score(y_val, np.argmax(val_pred, axis=1), average='weighted')
test_recall = recall_score(y_test, np.argmax(test_pred, axis=1), average='weighted')

train_f1 = f1_score(y_train, np.argmax(train_pred, axis=1), average='weighted')
val_f1 = f1_score(y_val, np.argmax(val_pred, axis=1), average='weighted')
test_f1 = f1_score(y_test, np.argmax(test_pred, axis=1), average='weighted')

train_roc_auc = roc_auc_score(y_train, train_pred, average='macro', multi_class='ovr')
val_roc_auc = roc_auc_score(y_val, val_pred, average='macro', multi_class='ovr')
test_roc_auc = roc_auc_score(y_test, test_pred, average='macro', multi_class='ovr')


In [18]:

# Print evaluation metrics
print("Train data Accuracy:", train_accuracy)
print("Train data Precision:", train_precision)
print("Train data Recall:", train_recall)
print("Train data F1 Score:", train_f1)
print("Train data ROC AUC Score:", train_roc_auc)

print("\nValidation data Accuracy:", val_accuracy)
print("Validation data Precision:", val_precision)
print("Validation data Recall:", val_recall)
print("Validation data F1 Score:", val_f1)
print("Validation data ROC AUC Score:", val_roc_auc)

print("\nTest data Accuracy:", test_accuracy)
print("Test data Precision:", test_precision)
print("Test data Recall:", test_recall)
print("Test data F1 Score:", test_f1)
print("Test data ROC AUC Score:", test_roc_auc)



Train data Accuracy: 0.9982817869415808
Train data Precision: 0.9982820750007694
Train data Recall: 0.9982817869415808
Train data F1 Score: 0.9982812977297294
Train data ROC AUC Score: 0.9999908254366403

Validation data Accuracy: 0.9202988792029888
Validation data Precision: 0.9199862098752922
Validation data Recall: 0.9202988792029888
Validation data F1 Score: 0.9197179329591705
Validation data ROC AUC Score: 0.9881941414191501

Test data Accuracy: 0.94375
Test data Precision: 0.9435801096107835
Test data Recall: 0.94375
Test data F1 Score: 0.9436153410402258
Test data ROC AUC Score: 0.9908833333333333


In [19]:
from tensorflow.keras.models import load_model
model.save('model_inception.h5')

In [20]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score

def calculate_metrics(y_true, y_pred, y_prob):
    accuracy = accuracy_score(y_true, y_pred)
    precision = precision_score(y_true, y_pred)
    recall = recall_score(y_true, y_pred)
    f1 = f1_score(y_true, y_pred)
    roc_auc = roc_auc_score(y_true, y_prob)
    return accuracy, precision, recall, f1, roc_auc

def print_metrics_table(train_metrics, val_metrics, test_metrics):
    print("Metrics\t\t\tTrain\t\tValidation\tTest")
    print("Accuracy\t\t{:.4f}\t\t{:.4f}\t\t{:.4f}".format(train_metrics[0], val_metrics[0], test_metrics[0]))
    print("Precision\t\t{:.4f}\t\t{:.4f}\t\t{:.4f}".format(train_metrics[1], val_metrics[1], test_metrics[1]))
    print("Recall\t\t\t{:.4f}\t\t{:.4f}\t\t{:.4f}".format(train_metrics[2], val_metrics[2], test_metrics[2]))
    print("F1 Score\t\t{:.4f}\t\t{:.4f}\t\t{:.4f}".format(train_metrics[3], val_metrics[3], test_metrics[3]))
    print("ROC AUC Score\t\t{:.4f}\t\t{:.4f}\t\t{:.4f}".format(train_metrics[4], val_metrics[4], test_metrics[4]))

# Assuming train_metrics, val_metrics, and test_metrics are tuples/lists containing metrics in the order: (accuracy, precision, recall, f1_score, roc_auc_score)
train_metrics = (train_accuracy, train_precision, train_recall, train_f1, train_roc_auc)
val_metrics = (val_accuracy, val_precision, val_recall, val_f1, val_roc_auc)
test_metrics = (test_accuracy, test_precision, test_recall, test_f1, test_roc_auc)

print_metrics_table(train_metrics, val_metrics, test_metrics)


Metrics			Train		Validation	Test
Accuracy		0.9983		0.9203		0.9437
Precision		0.9983		0.9200		0.9436
Recall			0.9983		0.9203		0.9437
F1 Score		0.9983		0.9197		0.9436
ROC AUC Score		1.0000		0.9882		0.9909
